### LECTURE & ECRITURE PANDAS <=> SQL

* énoncé:
  1. télécharger l'url et écrire en local si **ce n'est pas déjà fait**
     > ATTENTION encodage **iso-8859-1**, et séparateur ";"
  2. se donner un dataframe avec le "Nom de domaine" et le "Pays BE" et un millions de lignes
  3. renommer les colonnes du df comme les colonnes de la table domain_name
  4. créer la base de données dns.db depuis le fichier .sql
  5. connecter sqlaclhemy sur la bdd
  6. utiliser Dataframe.to_sql pour écrire dans la table domain_name
  7. lire table dans un dataframe

In [33]:
import os
import sqlite3
import pandas as pd
from time import time
import numpy as np

# pip install SQLAlchemy
# créer une connexion de bdd
from sqlalchemy import create_engine, Column
# pour créer / modifier un schéma de db
from sqlalchemy.types import Integer, String, Text, CHAR
from sqlalchemy.dialects.sqlite import insert
URL = "http://www.afnic.fr/wp-media/ftp/documentsOpenData/202105_OPENDATA_A-NomsDeDomaineEnPointFr.zip"

In [21]:
## téléchargement et écriture en local
encoding = "iso-8859-1"
if not os.path.exists("dns.zip"):
    pd.read_csv(URL, sep=";", encoding=encoding).to_csv(
        "dns.zip",
        encoding=encoding,
        compression={
            "method": "zip",
            "archive_name": "dns.csv"
        },
        index=False
    )

In [6]:
# élaguage
dns_df = pd.read_csv(
    "dns.zip", 
    encoding=encoding,
    usecols=["Nom de domaine", "Pays BE"],
    # 1M lignes après 1M en offset 
    nrows=1000000,
    # offset : int => depuis le "header" PB avec le usercols, range plsu malin
    skiprows=range(1, 1000000)
)
dns_df

,Nom de domaine,Pays BE
0,cabinet-leprovost.fr,FR
1,cabinet-lerasle.fr,FR
2,cabinetlerasle.fr,FR
3,cabinet-leray.fr,FR
4,cabinet-leray-lpa.fr,FR
...,...,...
999995,ecoledekunlun.fr,FR
999996,ecole-de-la-banque.fr,FR
999997,ecoledelabanque.fr,FR
999998,ecoledelabdomen.fr,FR


In [36]:
try:
    with sqlite3.connect("dns.db") as conn:
        cur = conn.cursor()
        with open("domain_name_sqlite3.sql", "r", encoding="utf8") as f:
            cur.executescript(f.read())
except (
    sqlite3.OperationalError, 
    sqlite3.DatabaseError,
    ConnectionError
) as e:
    print(e)

In [27]:
# connexion sqlAlchemy
pandas_conn = create_engine("sqlite:///dns.db")
pd.read_sql("pays", pandas_conn, index_col="iso2")
pd.read_sql("SELECT * FROM pays", pandas_conn, index_col="iso2")



,name
iso2,
AE,Emirats Arabes Unis
AT,Autriche
AU,Australie
BE,Belgique
BS,Bahamas
CA,Canada
CH,Chine
CY,Chypre
CZ,République tchèque


In [24]:
dns_df.rename(columns={
    "Nom de domaine": "name",
    "Pays BE": "iso2"
}, inplace=True)
dns_df

,name,iso2
0,cabinet-leprovost.fr,FR
1,cabinet-lerasle.fr,FR
2,cabinetlerasle.fr,FR
3,cabinet-leray.fr,FR
4,cabinet-leray-lpa.fr,FR
...,...,...
999995,ecoledekunlun.fr,FR
999996,ecole-de-la-banque.fr,FR
999997,ecoledelabanque.fr,FR
999998,ecoledelabdomen.fr,FR


In [39]:
### PAS avec SQLITE
# def insert_on_conflict_update(table, conn, keys, data_iter):
#     # update columns "b" and "c" on primary key conflict
#     data = [dict(zip(keys, row)) for row in data_iter]
#     stmt = (
#         insert(table.table)
#         .values(data)
#     )
#     print(type(stmt))
#     stmt = stmt.on_duplicate_key_update(b=stmt.inserted["dns_id"], c=stmt.inserted["dns_id"])
#     result = conn.execute(stmt)
#     return result.rowcount

print(dns_df.to_sql(
    "domain_name",
    pandas_conn,
    # if_exists="replace",
    ## essayer de créer
    # if_exists="fail",

    ## append (ajouter les lignes à la table existante)
    if_exists="append"
    ##  utilisation de l'index comme colonne
    index=True,
    index_label="dns_id",
    
    ## batch d'inserts
    chunksize=1000,
    ## surcharger le comportement du Insert => Insert or Update (pas avec SQLITE3)
    # method=insert_on_conflict_update
    
    ## en mode replace on écrase les données autant que le SCHEMA
    # or, les types SQLAlchemy ne peuvent pas spécifier les clés etc...
    #  donc surcharger pd.io.sql.PandasTable ATTENTION !!!
    ## uniquement en replace
    # dtype={
    #     "dns_id": Integer,
    #     "name": String(length=100),
    #     "iso2": CHAR(length=2),
    #     "created_at": Text
    # }
    
))

ValueError: Table 'domain_name' already exists.